# Zadatak klasifikacije teksta

Kao što smo spomenuli, fokusirat ćemo se na jednostavan zadatak klasifikacije teksta temeljen na **AG_NEWS** skupu podataka, gdje je cilj klasificirati naslove vijesti u jednu od 4 kategorije: Svijet, Sport, Poslovanje i Znanost/Tehnologija.

## Skup podataka

Ovaj skup podataka ugrađen je u modul [`torchtext`](https://github.com/pytorch/text), tako da mu možemo lako pristupiti.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Ovdje, `train_dataset` i `test_dataset` sadrže kolekcije koje vraćaju parove oznake (broj klase) i teksta, na primjer:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Dakle, ispisat ćemo prvih 10 novih naslova iz našeg skupa podataka:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Budući da su skupovi podataka iteratori, ako želimo koristiti podatke više puta, moramo ih pretvoriti u popis:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizacija

Sada trebamo pretvoriti tekst u **brojeve** koji se mogu predstaviti kao tenzori. Ako želimo reprezentaciju na razini riječi, trebamo učiniti dvije stvari:
* koristiti **tokenizator** za razdvajanje teksta na **tokene**
* izgraditi **vokabular** tih tokena.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Korištenjem rječnika, možemo lako kodirati naš tokenizirani niz u skup brojeva:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Predstavljanje teksta pomoću vreće riječi

Budući da riječi nose značenje, ponekad možemo shvatiti značenje teksta samo promatrajući pojedinačne riječi, bez obzira na njihov redoslijed u rečenici. Na primjer, prilikom klasifikacije vijesti, riječi poput *vrijeme*, *snijeg* vjerojatno ukazuju na *vremensku prognozu*, dok bi riječi poput *dionice*, *dolar* upućivale na *financijske vijesti*.

**Vreća riječi** (BoW) vektorsko predstavljanje najčešće je korišteno tradicionalno vektorsko predstavljanje. Svaka riječ povezana je s indeksom vektora, a element vektora sadrži broj pojavljivanja riječi u određenom dokumentu.

![Slika koja prikazuje kako je vektorsko predstavljanje vreće riječi prikazano u memoriji.](../../../../../translated_images/hr/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Napomena**: Možete također razmišljati o BoW kao o zbroju svih one-hot kodiranih vektora za pojedinačne riječi u tekstu.

Ispod je primjer kako generirati predstavljanje vreće riječi koristeći Scikit Learn python biblioteku:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Za izračunavanje vektora vreće riječi iz vektorske reprezentacije našeg AG_NEWS skupa podataka, možemo koristiti sljedeću funkciju:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Napomena:** Ovdje koristimo globalnu varijablu `vocab_size` za određivanje zadane veličine rječnika. Budući da je veličina rječnika često prilično velika, možemo ograničiti veličinu rječnika na najčešće riječi. Pokušajte smanjiti vrijednost `vocab_size` i pokrenuti donji kod te vidjeti kako to utječe na točnost. Možete očekivati određeni pad točnosti, ali ne dramatičan, u zamjenu za bolje performanse.


## Treniranje BoW klasifikatora

Sada kada smo naučili kako izgraditi Bag-of-Words reprezentaciju našeg teksta, trenirajmo klasifikator na temelju nje. Prvo, trebamo prilagoditi naš skup podataka za treniranje na način da se sve pozicijske vektorske reprezentacije pretvore u Bag-of-Words reprezentaciju. To se može postići prosljeđivanjem funkcije `bowify` kao parametra `collate_fn` standardnom torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Sada definirajmo jednostavnu neuronsku mrežu klasifikatora koja sadrži jedan linearni sloj. Veličina ulaznog vektora jednaka je `vocab_size`, a izlazna veličina odgovara broju klasa (4). Budući da rješavamo zadatak klasifikacije, konačna aktivacijska funkcija je `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Sada ćemo definirati standardnu PyTorch petlju za treniranje. Budući da je naš skup podataka prilično velik, za potrebe podučavanja trenirat ćemo samo jedan epoch, a ponekad čak i manje od jednog epocha (navođenjem parametra `epoch_size` možemo ograničiti treniranje). Također ćemo izvještavati o akumuliranoj točnosti treniranja tijekom treniranja; učestalost izvještavanja određuje se pomoću parametra `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrami, TriGrami i N-Grami

Jedno ograničenje pristupa vreće riječi je da su neke riječi dio višerječnih izraza, na primjer, riječ 'hot dog' ima potpuno drugačije značenje od riječi 'hot' i 'dog' u drugim kontekstima. Ako uvijek predstavljamo riječi 'hot' i 'dog' istim vektorima, to može zbuniti naš model.

Kako bismo to riješili, **N-gram reprezentacije** često se koriste u metodama klasifikacije dokumenata, gdje je učestalost svake riječi, dvostruke riječi ili trostruke riječi korisna značajka za treniranje klasifikatora. U bigram reprezentaciji, na primjer, dodajemo sve parove riječi u vokabular, uz originalne riječi.

Ispod je primjer kako generirati bigram reprezentaciju vreće riječi koristeći Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Glavni nedostatak N-gram pristupa je što veličina rječnika počinje rasti izuzetno brzo. U praksi, potrebno je kombinirati N-gram reprezentaciju s nekim tehnikama smanjenja dimenzionalnosti, poput *ugrađivanja* (embeddings), o čemu ćemo raspravljati u sljedećoj jedinici.

Kako bismo koristili N-gram reprezentaciju u našem **AG News** skupu podataka, trebamo izgraditi poseban ngram rječnik:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Mogli bismo koristiti isti kod kao gore za treniranje klasifikatora, no to bi bilo vrlo neučinkovito u pogledu memorije. U sljedećoj jedinici, trenirat ćemo bigram klasifikator koristeći ugradnje.

> **Napomena:** Možete zadržati samo one n-grame koji se pojavljuju u tekstu više od određenog broja puta. Ovo će osigurati da se rijetki bigrami izostave i značajno smanjiti dimenzionalnost. Da biste to postigli, postavite parametar `min_freq` na višu vrijednost i promatrajte kako se duljina rječnika mijenja.


## Frekvencija termina i inverzna frekvencija dokumenata (TF-IDF)

U BoW (Bag of Words) reprezentaciji, pojavljivanja riječi imaju jednaku težinu, bez obzira na samu riječ. Međutim, jasno je da su učestale riječi, poput *a*, *u*, itd., mnogo manje važne za klasifikaciju od specijaliziranih pojmova. Zapravo, u većini NLP zadataka neke su riječi relevantnije od drugih.

**TF-IDF** označava **frekvenciju termina – inverznu frekvenciju dokumenata**. To je varijacija metode vreće riječi, gdje se umjesto binarne vrijednosti 0/1, koja označava pojavljivanje riječi u dokumentu, koristi vrijednost s pomičnim zarezom, koja je povezana s učestalošću pojavljivanja riječi u korpusu.

Formalnije, težina $w_{ij}$ riječi $i$ u dokumentu $j$ definirana je kao:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
gdje su:
* $tf_{ij}$ broj pojavljivanja riječi $i$ u dokumentu $j$, tj. BoW vrijednost koju smo ranije vidjeli
* $N$ broj dokumenata u zbirci
* $df_i$ broj dokumenata koji sadrže riječ $i$ u cijeloj zbirci

TF-IDF vrijednost $w_{ij}$ raste proporcionalno broju pojavljivanja riječi u dokumentu, ali se smanjuje u odnosu na broj dokumenata u korpusu koji sadrže tu riječ, što pomaže u prilagodbi činjenici da se neke riječi pojavljuju češće od drugih. Na primjer, ako se riječ pojavljuje u *svakom* dokumentu u zbirci, tada je $df_i=N$, i $w_{ij}=0$, te bi ti termini bili potpuno zanemareni.

TF-IDF vektorizaciju teksta možete jednostavno kreirati pomoću Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Zaključak

Iako TF-IDF reprezentacije dodjeljuju težinu učestalosti različitim riječima, one nisu sposobne prikazati značenje ili redoslijed. Kao što je poznati lingvist J. R. Firth rekao 1935. godine: "Potpuno značenje riječi uvijek je kontekstualno, i nijedno proučavanje značenja izvan konteksta ne može se smatrati ozbiljnim." Kasnije u ovom tečaju naučit ćemo kako izvući kontekstualne informacije iz teksta koristeći jezično modeliranje.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
